In [ ]:
# Copyright 2021 NVIDIA CORPORATION
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
n_files = 40 
n_bags = 1

In [2]:
import glob
import pandas as pd
import numpy as np
import cudf
import cupy
import gc

from datetime import datetime

In [3]:
files = sorted(glob.glob('/raid/recsys2021_pre_TE_2/*'))[0:(n_files//2)]+sorted(glob.glob('/raid/recsys2021_pre_TE_3/*'))[0:(n_files//2)]

#files = sorted(glob.glob('/raid/recsys2021_pre_TE_submission_2/*'))[0:(n_files//2)]+sorted(glob.glob('/raid/recsys2021_pre_TE_submission_3/*'))[0:(n_files//2)]

In [4]:
files

['/raid/recsys2021_pre_TE_2/part-00000.parquet',
 '/raid/recsys2021_pre_TE_2/part-00001.parquet',
 '/raid/recsys2021_pre_TE_2/part-00002.parquet',
 '/raid/recsys2021_pre_TE_2/part-00003.parquet',
 '/raid/recsys2021_pre_TE_2/part-00004.parquet',
 '/raid/recsys2021_pre_TE_2/part-00005.parquet',
 '/raid/recsys2021_pre_TE_2/part-00006.parquet',
 '/raid/recsys2021_pre_TE_2/part-00007.parquet',
 '/raid/recsys2021_pre_TE_2/part-00008.parquet',
 '/raid/recsys2021_pre_TE_2/part-00009.parquet',
 '/raid/recsys2021_pre_TE_2/part-00010.parquet',
 '/raid/recsys2021_pre_TE_2/part-00011.parquet',
 '/raid/recsys2021_pre_TE_2/part-00012.parquet',
 '/raid/recsys2021_pre_TE_2/part-00013.parquet',
 '/raid/recsys2021_pre_TE_2/part-00014.parquet',
 '/raid/recsys2021_pre_TE_2/part-00015.parquet',
 '/raid/recsys2021_pre_TE_2/part-00016.parquet',
 '/raid/recsys2021_pre_TE_2/part-00017.parquet',
 '/raid/recsys2021_pre_TE_2/part-00018.parquet',
 '/raid/recsys2021_pre_TE_2/part-00019.parquet',
 '/raid/recsys2021_p

In [5]:
import os, time
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3,4,5,6,7"

In [6]:
import dask as dask, dask_cudf
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
import subprocess

In [7]:
cluster = LocalCUDACluster(local_directory='/raid/dask3/', device_memory_limit=0.5)
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:37535 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 8 Memory: 429.50 GB


In [8]:
train = dask_cudf.read_parquet(sorted(files))

In [9]:
train, = dask.persist(train)

In [10]:
label_names = ['reply', 'retweet', 'retweet_comment', 'like']
DONT_USE = ['timestamp','a_account_creation','b_account_creation','engage_time',
            'fold','b_user_id','a_user_id', 'dt_dow', 'a_account_creation', 
            'b_account_creation', 'elapsed_time', 'links','domains','hashtags','id', 'date', 'is_train', 
            'tw_hash0', 'tw_hash1', 'tw_hash2', 'tw_http0', 'tw_uhash', 'tw_hash', 'tw_word0', 
            'tw_word1', 'tw_word3', 'tw_word4', 'dt_minute', 'dt_second']
DONT_USE += label_names
features = [c for c in train.columns if c not in DONT_USE]

RMV = [c for c in DONT_USE if c in train.columns and c not in label_names]
RMV = list(set(RMV))

In [11]:
%%time

train = train.drop(RMV,axis=1)
wait(train)

CPU times: user 868 ms, sys: 232 ms, total: 1.1 s
Wall time: 6.62 s


DoneAndNotDoneFutures(done={<Future: finished, type: cudf.DataFrame, key: ('read-parquet-a880c4763b081fdeeafe58dff7a31bd8', 37)>, <Future: finished, type: cudf.DataFrame, key: ('read-parquet-a880c4763b081fdeeafe58dff7a31bd8', 20)>, <Future: finished, type: cudf.DataFrame, key: ('read-parquet-a880c4763b081fdeeafe58dff7a31bd8', 3)>, <Future: finished, type: cudf.DataFrame, key: ('read-parquet-a880c4763b081fdeeafe58dff7a31bd8', 0)>, <Future: finished, type: cudf.DataFrame, key: ('read-parquet-a880c4763b081fdeeafe58dff7a31bd8', 23)>, <Future: finished, type: cudf.DataFrame, key: ('read-parquet-a880c4763b081fdeeafe58dff7a31bd8', 6)>, <Future: finished, type: cudf.DataFrame, key: ('read-parquet-a880c4763b081fdeeafe58dff7a31bd8', 12)>, <Future: finished, type: cudf.DataFrame, key: ('read-parquet-a880c4763b081fdeeafe58dff7a31bd8', 9)>, <Future: finished, type: cudf.DataFrame, key: ('read-parquet-a880c4763b081fdeeafe58dff7a31bd8', 35)>, <Future: finished, type: cudf.DataFrame, key: ('read-parqu

In [12]:
Y_train = train[label_names]
Y_train, = dask.persist(Y_train)
Y_train.head()    
    
train = train.drop(['tweet_id']+label_names,axis=1)
train, = dask.persist(train)
train.head()


features = [c for c in train.columns if c not in DONT_USE]
print('Using %i features:'%(len(features)),train.shape[1])
np.asarray(features)

Using 84 features: 84


array(['TE_a_user_id_like', 'TE_a_user_id_reply', 'TE_a_user_id_retweet',
       'TE_a_user_id_retweet_comment', 'TE_b_is_verified_tweet_type_like',
       'TE_b_is_verified_tweet_type_reply',
       'TE_b_is_verified_tweet_type_retweet',
       'TE_b_is_verified_tweet_type_retweet_comment',
       'TE_b_user_id_a_user_id_like', 'TE_b_user_id_a_user_id_reply',
       'TE_b_user_id_a_user_id_retweet',
       'TE_b_user_id_a_user_id_retweet_comment', 'TE_b_user_id_like',
       'TE_b_user_id_reply', 'TE_b_user_id_retweet',
       'TE_b_user_id_retweet_comment',
       'TE_b_user_id_tweet_type_language_like',
       'TE_b_user_id_tweet_type_language_reply',
       'TE_b_user_id_tweet_type_language_retweet',
       'TE_b_user_id_tweet_type_language_retweet_comment',
       'TE_domains_language_b_follows_a_tweet_type_media_a_is_verified_like',
       'TE_domains_language_b_follows_a_tweet_type_media_a_is_verified_reply',
       'TE_domains_language_b_follows_a_tweet_type_media_a_is_verified

In [13]:
import xgboost as xgb
print('XGB Version',xgb.__version__)

xgb_parms = { 
    'max_depth':8, 
    'learning_rate':0.1, 
    'subsample':0.8,
    'colsample_bytree':0.3, 
    'eval_metric':'logloss',
    'objective':'binary:logistic',
    'tree_method':'gpu_hist',
    'predictor' : 'gpu_predictor'
}

XGB Version 1.1.0


In [14]:
for col in train.columns:
    if train[col].dtype=='bool':
        train[col] = train[col].astype('int8')
train, = dask.persist(train)
train.head()

,TE_a_user_id_like,TE_a_user_id_reply,TE_a_user_id_retweet,TE_a_user_id_retweet_comment,TE_b_is_verified_tweet_type_like,TE_b_is_verified_tweet_type_reply,TE_b_is_verified_tweet_type_retweet,TE_b_is_verified_tweet_type_retweet_comment,TE_b_user_id_a_user_id_like,TE_b_user_id_a_user_id_reply,...,tw_last_quest,tw_len_gif,tw_len_media,tw_len_photo,tw_len_quest,tw_len_retweet,tw_len_rt,tw_len_token,tw_len_video,tweet_type
2,0.397,0.016,0.121,0.001,0.485,0.040,0.078,0.008,0.479,0.021,...,0,1,1,0,0,0,0,40,0,0
3,0.725,0.012,0.063,0.013,0.485,0.040,0.078,0.008,0.518,0.023,...,0,0,0,0,0,0,0,40,0,0
4,0.397,0.057,0.072,0.002,0.485,0.040,0.078,0.008,0.546,0.115,...,0,0,0,0,0,0,0,34,0,0
5,0.397,0.032,0.022,0.002,0.485,0.040,0.078,0.008,0.452,0.026,...,0,0,0,0,0,0,0,2,0,0
6,0.397,0.034,0.260,0.002,0.247,0.006,0.113,0.006,0.345,0.025,...,0,0,3,3,0,0,0,24,0,1


In [15]:
import pickle

In [16]:
models_index = [0,1,2,3]

In [17]:
def get_colnames(train, name):
    cols = list(train.columns)
    cols = [col for col in cols if 'NN_' not in col and col!='group' and cols!='quantile']
    cols = [x for x in cols if ('TE_' in x and 'a_user_id' in x and name in x) or 'TE_' not in x or ('TE_' in x and 'a_user_id' not in x)]
    cols = cols + [x for x in train.columns if x not in cols and 'TE_switch_' in x]
    return(cols)

In [18]:
%%time
# TRAIN AND VALIDATE

NROUND = [[642], [998], [479], [483]]
VERBOSE_EVAL = 50
ESR = 50
preds_out = []

    
for i in models_index:
    name = label_names[i]
    print('#'*25);print('###',name);print('#'*25)
    cols = get_colnames(train, name)  
    start = time.time(); print('Creating DMatrix...')
    dtrain = xgb.dask.DaskDMatrix(client,data=train[cols].values,label=Y_train.iloc[:, i])
    print('Took %.1f seconds'%(time.time()-start))
    print(len(cols))
    print(cols)
    xgb_parms['seed'] = 0
    start = time.time(); print('Training...')
    model = xgb.dask.train(client, xgb_parms, 
                           dtrain=dtrain,
                           evals=[(dtrain,'valid')],
                           num_boost_round=NROUND[i][0],
                           verbose_eval=VERBOSE_EVAL
                          ) 
    print('Took %.1f seconds'%(time.time()-start))
    logloss_valid = model["history"]['valid']['logloss'][::VERBOSE_EVAL]
    for ik in range(len(logloss_valid)):
        print(f"{str(VERBOSE_EVAL*ik).zfill(4)} valid-logloss:{logloss_valid[ik]:.5f}")

    start = time.time(); print('Predicting...')
    pickle.dump(model, open('model_' + str(name) + '.pickle', 'wb'))
    print('Took %.1f seconds'%(time.time()-start))

#########################
### reply
#########################
Creating DMatrix...
Took 4.0 seconds
77
['TE_a_user_id_reply', 'TE_b_is_verified_tweet_type_like', 'TE_b_is_verified_tweet_type_reply', 'TE_b_is_verified_tweet_type_retweet', 'TE_b_is_verified_tweet_type_retweet_comment', 'TE_b_user_id_a_user_id_reply', 'TE_b_user_id_like', 'TE_b_user_id_reply', 'TE_b_user_id_retweet', 'TE_b_user_id_retweet_comment', 'TE_b_user_id_tweet_type_language_like', 'TE_b_user_id_tweet_type_language_reply', 'TE_b_user_id_tweet_type_language_retweet', 'TE_b_user_id_tweet_type_language_retweet_comment', 'TE_domains_language_b_follows_a_tweet_type_media_a_is_verified_like', 'TE_domains_language_b_follows_a_tweet_type_media_a_is_verified_reply', 'TE_domains_language_b_follows_a_tweet_type_media_a_is_verified_retweet', 'TE_domains_language_b_follows_a_tweet_type_media_a_is_verified_retweet_comment', 'TE_media_tweet_type_language_a_is_verified_b_is_verified_b_follows_a_like', 'TE_media_tweet_type_language_

Took 81.8 seconds
0000 valid-logloss:0.60070
0050 valid-logloss:0.03555
0100 valid-logloss:0.03326
0150 valid-logloss:0.03288
0200 valid-logloss:0.03264
0250 valid-logloss:0.03245
0300 valid-logloss:0.03229
0350 valid-logloss:0.03216
0400 valid-logloss:0.03202
0450 valid-logloss:0.03189
Predicting...
Took 0.3 seconds
#########################
### like
#########################
Creating DMatrix...
Took 32.1 seconds
77
['TE_a_user_id_like', 'TE_b_is_verified_tweet_type_like', 'TE_b_is_verified_tweet_type_reply', 'TE_b_is_verified_tweet_type_retweet', 'TE_b_is_verified_tweet_type_retweet_comment', 'TE_b_user_id_a_user_id_like', 'TE_b_user_id_like', 'TE_b_user_id_reply', 'TE_b_user_id_retweet', 'TE_b_user_id_retweet_comment', 'TE_b_user_id_tweet_type_language_like', 'TE_b_user_id_tweet_type_language_reply', 'TE_b_user_id_tweet_type_language_retweet', 'TE_b_user_id_tweet_type_language_retweet_comment', 'TE_domains_language_b_follows_a_tweet_type_media_a_is_verified_like', 'TE_domains_langua

In [19]:
client.close()

In [20]:
files = glob.glob('./model_*.pickle')
files = [x for x in files if '_valid' not in x and '_pred' not in x]

In [21]:
len(files), files

(4,
 ['./model_retweet_comment.pickle',
  './model_retweet.pickle',
  './model_like.pickle',
  './model_reply.pickle'])

In [22]:
import os

!rm ./models/
!mkdir ./models/
for file in files:
    os.system('mv ' + file + ' ./models/')

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
rm: cannot remove './models/': No such file or directory
/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [23]:
!ls ./models/

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
model_like.pickle   model_retweet.pickle
model_reply.pickle  model_retweet_comment.pickle


In [24]:
!rm models.zip

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
rm: cannot remove 'models.zip': No such file or directory


In [25]:
!zip -r models.zip models

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
  adding: models/ (stored 0%)
  adding: models/model_retweet_comment.pickle (deflated 50%)
  adding: models/model_retweet.pickle (deflated 50%)
  adding: models/model_like.pickle (deflated 50%)
  adding: models/model_reply.pickle (deflated 50%)


In [26]:
client.close()